# Select unlabeled documents

In [6]:
import os
import shutil

In [7]:
db_path = "../../Datasets/Tobacco 800 Dataset/tobacco800"
annotated_images = "../../Old_Data/Annotated_Images"
ann_img_list = os.listdir(annotated_images)
images_list = os.listdir(db_path)
print(len(ann_img_list))
print(len(images_list))

## Remove annotated images ##
for img_f in ann_img_list:
    file = img_f+".png"
    images_list.remove(file)

print(len(images_list))


src_path = db_path
dst_path = "../../Data/Images_2_Annotate/"

for file in images_list:
    src_file = os.path.join(src_path, file)
    dst_file = os.path.join(dst_path, file)
    shutil.copy2(src_file, dst_file)



500
1292
792


# Get Ground truth anchor boxes

In [ ]:
import os
import numpy as np

def load_yolo_annotations(annotation_file):
    """
    Load YOLOv5 annotations from a file.
    """
    bboxes = []
    with open(annotation_file, 'r') as file:
        lines = file.readlines()
        for line in lines:
            _, _, _, width, height = map(float, line.strip().split())
            bboxes.append((width, height))
    return bboxes

def get_ground_truth_bboxes(dataset_path):
    """
    Extract ground truth bounding box widths and heights from a YOLOv5 formatted dataset.
    
    Parameters:
    - dataset_path: Path to the dataset directory containing the 'train' folder with images and annotations.
    
    Returns:
    - bboxes: List of tuples containing the width and height of each bounding box.
    """
    train_path = os.path.join(dataset_path, 'train')
    annotations_path = os.path.join(train_path, 'labels')
    
    bboxes = []
    
    for annotation_file in os.listdir(annotations_path):
        if annotation_file.endswith('.txt'):
            annotation_file_path = os.path.join(annotations_path, annotation_file)
            bboxes += load_yolo_annotations(annotation_file_path)
    
    return bboxes

# Example usage
dataset_path = 'path/to/your/yolov5/dataset'
bboxes = get_ground_truth_bboxes(dataset_path)

# Print the first 10 bounding boxes
for bbox in bboxes[:10]:
    print(f"Width: {bbox[0]}, Height: {bbox[1]}")


# Define Anchors

In [5]:
import numpy as np
from sklearn.cluster import KMeans

def kmeans_anchors(bboxes, num_anchors):
    kmeans = KMeans(n_clusters=num_anchors, random_state=0).fit(bboxes)
    anchors = kmeans.cluster_centers_
    return anchors

# Example ground truth bounding boxes (width, height)
bboxes = np.array([
    [30, 40],
    [50, 80],
    [100, 120],
    [200, 300],
    [60, 45],
    [120, 160],
    [160, 200],
    [320, 320],
    [400, 500],
    # Add more bounding boxes from your dataset
])

# Perform k-means clustering to find the anchor boxes
num_anchors = 3  # Total number of anchors
anchors = kmeans_anchors(bboxes, num_anchors)

# Define the scaling factors for each stage
scale_factor_p3 = 8
scale_factor_p4 = 16
scale_factor_p5 = 32

# Scale all anchors according to each feature map size
anchors_in = [(round(w,2) , round(h,2) ) for w, h in anchors]
anchors_p3 = [(round(w / scale_factor_p3, 2), round(h / scale_factor_p3, 2)) for w, h in anchors]
anchors_p4 = [(round(w / scale_factor_p4, 2), round(h / scale_factor_p4, 2)) for w, h in anchors]
anchors_p5 = [(round(w / scale_factor_p5, 2), round(h / scale_factor_p5, 2)) for w, h in anchors]
# Print the scaled anchors
print("Anchors for (640x640):", anchors_in)
print("Anchors for P3/8:", anchors_p3)
print("Anchors for P4/16:", anchors_p4)
print("Anchors for P5/32:", anchors_p5)



Anchors for (640x640): [(72.0, 89.0), (400.0, 500.0), (226.67, 273.33)]
Anchors for P3/8: [(9.0, 11.12), (50.0, 62.5), (28.33, 34.17)]
Anchors for P4/16: [(4.5, 5.56), (25.0, 31.25), (14.17, 17.08)]
Anchors for P5/32: [(2.25, 2.78), (12.5, 15.62), (7.08, 8.54)]
